# __Word to Word example__

Code from https://datascience.stackexchange.com/questions/40038/how-to-implement-word-to-word-co-occurence-matrix-in-python

Data from https://archive.ics.uci.edu/ml/datasets/Amazon+Commerce+reviews+set#

In [1]:
import numpy as np
import pandas as pd
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import combinations
from collections import Counter

## Example 1

In [2]:
ctxs = [
    'krayyem like candy crush more then coffe',
    'krayyem plays candy crush all days',
    'krayyem do not invite his friends to play candy crush',
    'krayyem is smart',
]

l_unique = list(set((' '.join(ctxs)).split(' ')))
mat = np.zeros((len(l_unique), len(l_unique)))

nei = []
nei_size = 3

for ctx in ctxs:
    for word in ctx.split(' '):
        nei.append(word)
        if len(nei) > nei_size:
            nei.pop(0)
        for word_1 in nei:
            for word_2 in nei:
                # if word_1 != word_2 -> to avoid diagonal
                mat[l_unique.index(word_1), l_unique.index(word_2)] += 1

mat = pd.DataFrame(mat)
mat.index = l_unique
mat.columns = l_unique
display(mat)

,like,more,all,invite,crush,not,his,days,friends,do,to,play,smart,krayyem,coffe,is,then,candy,plays
like,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0
more,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0
all,0.0,0.0,3.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
invite,0.0,0.0,0.0,3.0,0.0,2.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
crush,1.0,2.0,2.0,0.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,6.0,1.0
not,0.0,0.0,0.0,2.0,0.0,3.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
his,0.0,0.0,0.0,2.0,0.0,1.0,3.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
days,0.0,0.0,2.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
friends,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
do,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


## Example 2

In [3]:
sentences = ['i go to london', 'you do not go to london', 'but london goes to you']
vocab = set(word_tokenize(' '.join(sentences)))
print('Vocabulary:\n',vocab,'\n')
token_sent_list = [word_tokenize(sen) for sen in sentences]
print('Each sentence in token form:\n',token_sent_list,'\n')

co_occ = {ii:Counter({jj:0 for jj in vocab if jj!=ii}) for ii in vocab}
k=2

for sen in token_sent_list:
    for ii in range(len(sen)):
        if ii < k:
            c = Counter(sen[0:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        elif ii > len(sen)-(k+1):
            c = Counter(sen[ii-k::])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        else:
            c = Counter(sen[ii-k:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c

# Having final matrix in dict form lets you convert it to different python data structures
co_occ = {ii:dict(co_occ[ii]) for ii in vocab}
display(co_occ)

Vocabulary:
 {'not', 'to', 'goes', 'do', 'go', 'but', 'london', 'i', 'you'} 

Each sentence in token form:
 [['i', 'go', 'to', 'london'], ['you', 'do', 'not', 'go', 'to', 'london'], ['but', 'london', 'goes', 'to', 'you']] 



{'not': {'to': 1, 'do': 1, 'go': 1, 'you': 1},
 'to': {'go': 2, 'london': 3, 'i': 1, 'not': 1, 'goes': 1, 'you': 1},
 'goes': {'to': 1, 'but': 1, 'london': 1, 'you': 1},
 'do': {'not': 1, 'go': 1, 'you': 1},
 'go': {'to': 2, 'london': 2, 'i': 1, 'do': 1, 'not': 1},
 'but': {'goes': 1, 'london': 1},
 'london': {'to': 3, 'go': 2, 'but': 1, 'goes': 1},
 'i': {'to': 1, 'go': 1},
 'you': {'not': 1, 'do': 1, 'goes': 1, 'to': 1}}

## USING A DATASET

In [4]:
# data from: https://data.world/ostp/wsrd-testbed-inventory 
data_inventory = pd.read_csv('data/Testbed_Inventory.csv')

In [54]:
# Remove space at the end of the sentence
data_world = data_inventory['name'].str.replace(r' $', '')
# Create a list of the sentences
text = list(data_world)

In [55]:
# Remove punctuation 
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in text]
# Converting the case to lower
lowered_text = [word.lower() for word in  stripped]
print(lowered_text)

['calit2 ericsson wireless access network research testbed', 'afrl aerial layer networking facilities', 'nrl cognitive radio test laboratory', 'orbit open access next generation wireless network testbed', 'spectrum sharing innovation testbed', 'public safety communications research lab pscr', 'cognitive radio network testbed cornet', 'nrl tactical edge network testbed', 'ornl communications testbed', 'electronic proving ground us army test ranges', 'white sands missile range us army test ranges', 'aberdeen test center us army test ranges', 'redstone test center us army test ranges', 'yuma proving ground us army test ranges', 'idaho national laboratory wireless national testbed', 'cmulab', 'd meas learn', 'digital object registry', 'cloudctl vise', 'bgpmux dtunnels', 'enterprisegeni openflow', 'geni4yr', 'gmoc netkarma kgeni', 'gpeni', 'gush proto', 'instools ism infrastructure', 'kansei otm', 'max', 'measurementsys', 'millionnodegeni security', 'orbit wimax', 'orcaben', 'planetlab scaf

In [56]:
# Removing STOP WORDS
set(stopwords.words('english'))
stop_words = stopwords.words('english')
stop_words.append('calit2')
filtered_sentence = [w for w in lowered_text if not w in stop_words] 

In [52]:
lowered_text

['afrl aerial layer networking facilities',
 'nrl cognitive radio test laboratory',
 'orbit open access next generation wireless network testbed',
 'spectrum sharing innovation testbed',
 'public safety communications research lab pscr',
 'cognitive radio network testbed cornet',
 'nrl tactical edge network testbed',
 'ornl communications testbed',
 'electronic proving ground us army test ranges',
 'white sands missile range us army test ranges',
 'aberdeen test center us army test ranges',
 'redstone test center us army test ranges',
 'yuma proving ground us army test ranges',
 'idaho national laboratory wireless national testbed',
 'cmulab',
 'd meas learn',
 'digital object registry',
 'cloudctl vise',
 'bgpmux dtunnels',
 'enterprisegeni openflow',
 'geni4yr',
 'gmoc netkarma kgeni',
 'gpeni',
 'gush proto',
 'instools ism infrastructure',
 'kansei otm',
 'max',
 'measurementsys',
 'millionnodegeni security',
 'orbit wimax',
 'orcaben',
 'planetlab scaffold federation',
 'protogeni

### Example 1

In [ ]:
text_list = filtered_sentence
l_unique = list(set((' '.join(text_list)).split(' ')))
mat = np.zeros((len(l_unique), len(l_unique)))
nei = []
nei_size = 3

for ctx in text_list:
    for word in ctx.split(' '):
        nei.append(word)
        if len(nei) > nei_size:
            nei.pop(0)
        for word_1 in nei:
            for word_2 in nei:
                # if word_1 != word_2 -> to avoid diagonal
                mat[l_unique.index(word_1), l_unique.index(word_2)] += 1

mat = pd.DataFrame(mat)
mat.index = l_unique
mat.columns = l_unique
display(mat)

In [ ]:
sentences = filtered_sentence
vocab = set(word_tokenize(' '.join(sentences)))
#print('Vocabulary:\n',vocab,'\n')
token_sent_list = [word_tokenize(sen) for sen in sentences]
#print('Each sentence in token form:\n',token_sent_list,'\n')

co_occ = {ii:Counter({jj:0 for jj in vocab if jj!=ii}) for ii in vocab}
k=2

for sen in token_sent_list:
    for ii in range(len(sen)):
        if ii < k:
            c = Counter(sen[0:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        elif ii > len(sen)-(k+1):
            c = Counter(sen[ii-k::])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        else:
            c = Counter(sen[ii-k:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c

# Having final matrix in dict form lets you convert it to different python data structures
co_occ = {ii:dict(co_occ[ii]) for ii in vocab}
display(co_occ)